# Demo ConvNet para identificar LETRAS & NÚMEROS (realiza el entrenamiento y validación del modelo)
Se usa el dataset de imágenes obtenido de http://www.ee.surrey.ac.uk/CVSSP/demos/chars74k/

1) Cargar librerías:

In [0]:
# nota se debe indicar la versión 1 de TF para compatibilidad del código
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from IPython.display import Image

import os
import os.path

import json
from keras.models import model_from_json
from keras.models import load_model

import numpy as np 
import matplotlib.pyplot as plt

print ("Librerías cargadas.")

2) Cargar imágenes a procesar:

In [0]:
# monta Google Drive:
# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive')

# directorio local en Google Drive
path = 'gdrive/My Drive/IA/demoConvNet-Letras'

# define los nombres de los archivos a utilizar para leer/grabar el modelo
history_file_name = path + '/Model/CNN_L_history_dump_final.json'
weights_file_name = path + '/Model/CNN_L_model_final.h5'
model_json_file_name = path + '/Model/CNN_L_model_final.json'

In [0]:
# ajusta el tamaño para achicar los valores del RGB (que pasen a ser de 0 a 1 en vez de 0 a 255)
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = False)

test_datagen = ImageDataGenerator(rescale = 1./255)

# también se resizean las imagenes a 128x128
target_size = (128, 128)
batch_size = 500

# levanta el nombre de las clases
all_classes = os.listdir("".join([path, '/Letras/train/']))
print("Clases ",  len(all_classes), ": ", all_classes)

# levanta imágenes de entrenamiento
print("\nImágenes de entrenamiento: ")
training_set = train_datagen.flow_from_directory("".join([path, '/Letras/train/']), 
                                                 target_size=target_size, 
                                                 batch_size=batch_size, 
                                                 shuffle=True, 
                                                 classes=all_classes,
                                                 class_mode="categorical")
# levanta imágenes de prueba
print("\nImágenes de prueba: ")
test_set = test_datagen.flow_from_directory("".join([path, '/Letras/test/']), 
                                            target_size=target_size, 
                                            batch_size=batch_size,
                                            classes=all_classes,
                                            shuffle=True, 
                                            class_mode="categorical")
#print(training_set.class_indices)

3) Construye el modelo:

- Define el modelo de cero (primera ejecución):

In [0]:
# construye modelo de la nueva ConvNet
classifier = Sequential()
classifier.add(Conv2D(64, (3, 3), input_shape = (128, 128, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Conv2D(64, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Flatten())

classifier.add(Dense(units = 64, activation = 'relu'))
classifier.add(Dropout(0.5))

# genera una salida softmax por clase de imágenes detectada
classifier.add(Dense(units = len(training_set.class_indices), activation='softmax'))


# compila el modelo
classifier.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# muestra el modelo creado
print(classifier.summary())

- Carga un modelo ya pre-entrenado (luego de primera ejecución):

In [0]:
# carga modelo ya grabado de ConvNet
if os.path.isfile(model_json_file_name):
    classifier = load_model(weights_file_name)

    if os.path.isfile(history_file_name):
      h = json.load(open(history_file_name, 'r'))
      print("Modelo cargado: [", weights_file_name, "], [", history_file_name, "] y [", model_json_file_name, "] ")
    else: 
      print("No se encuentra modelo para cargar")
else:   
    print("No se encuentra modelo para cargar")

# muestra el modelo cargado
print(classifier.summary())

4) Entrenar el modelo:

In [0]:
# parámetros
steps_per_epoch = 25 #50
epochs = 5
validation_steps = 15

# define funciones especiales de "callbacks" 
# que se ejecutan durante el entrenamiento
# para terminar antes si el accuracy no mejora
# y guardar mejores modelos generados (menor loss)
early_stop = EarlyStopping(monitor='acc',  # 'loss'
                           min_delta=0.001, 
                           patience=3, 
                           mode='max', # 'min'
                           verbose=1)
checkpoint = ModelCheckpoint(weights_file_name, 
                             monitor='loss',                              
                             verbose=1, 
                             save_best_only=True, 
                             mode='min', 
                             period=1)

# manda a entrenar el modelo (conviene usar GPU)
history = classifier.fit_generator(training_set, 
                      steps_per_epoch = steps_per_epoch, 
                      epochs = epochs, 
                      validation_data = test_set, 
                      validation_steps = validation_steps,
                      callbacks = [early_stop,checkpoint])

h = history.history

4') Graba el modelo entrenado:



In [0]:
# graba el modelo

#classifier.save(weights_file_name) -- Se graba al hacer el entrenamiento

model_json = classifier.to_json()
with open(model_json_file_name, "w") as json_file:
    json_file.write(model_json)

with open(history_file_name, 'w') as f:
  json.dump(h, f)

print("Modelo grabado: [", weights_file_name, "], [", history_file_name, "] y [", model_json_file_name, "] ")


5) Muestra estadísticas y resultados del probar el modelo:

In [0]:
# muestra gráfico de exactitud y loss
plt.plot(h['acc'])
plt.plot(h['val_acc'])
plt.title('Exactitud del Modelo')
plt.ylabel('Exactitud')
plt.xlabel('Época')
plt.legend(['Entrenamiento', 'Validación'], loc='upper left')
plt.show()

plt.plot(h['loss'])
plt.plot(h['val_loss'])
plt.title('Error del Modelo')
plt.ylabel('Loss')
plt.xlabel('Época')
plt.legend(['Entrenamiento', 'Validación'], loc='upper left')
plt.show()


In [0]:
from sklearn.metrics.classification import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# define función auxiliar para mostrar resultado de cada imágen
def testImage(file_name, image_sample, classDesired, showPredictOK, showPredictError):
    result = classifier.predict(image_sample)
    
    # identifica mejor
    bestPos = np.argmax(result, axis=1)
    clasPred = str(all_classes[int(bestPos)])

    prediction = "".join([clasPred, "", str(result[0][bestPos]), "" ])
    if clasPred == classDesired:
      res = True      
    else:
      res = False
      prediction = "".join([prediction, "!"])

    # muestra resultados (solo con error)
    if (((not res) and showPredictError) or showPredictOK):
      print("> ", file_name,": " , prediction)
      img = Image(file_name, width = "80", height = "50")
      display(img)

    return res, clasPred

# define función auxiliar para mostrar resultado de cada directorio
def testAllClass(classDesired):
  cantOK = 0
  cantNOK = 0
  predict_path = "".join([path, '/Letras/test/', str(classDesired)])
  print("\n")
  y_classReal = []
  y_classRes = []
  for file in os.listdir(predict_path):
      if not file.startswith('.'):
          file = predict_path + "/" + file

          image_sample = image.load_img(file, target_size = (128, 128))
          image_sample = image.img_to_array(image_sample)
          image_sample = np.expand_dims(image_sample, axis = 0)
          
          result, clRes = testImage(file, image_sample, classDesired, False, False)
          if (result):
            cantOK = cantOK + 1
          else:
            cantNOK = cantNOK + 1

          y_classReal.append(classDesired)
          y_classRes.append(clRes)

  print("\nTOTAL CLASS", classDesired,": ", cantOK+cantNOK, ": Detectado OK ", cantOK, "imágenes - Detectado con Error ", cantNOK, "imágenes.")  
  print('con una Exactitud de %f' % accuracy_score(y_classReal, y_classRes))

  return cantOK, cantNOK, y_classReal, y_classRes

# procesa las imágenes de la carpeta <Test>
y_tests = []
y_preds = []
okGral = 0 
NokGral = 0
all_dirs = os.listdir("".join([path, '/Letras/test']))
for each_dir in all_dirs:
  print("\n--- Procesando ", each_dir)
  ok, nok, tests, preds = testAllClass(each_dir)
  print("\n--------------------------------------------------------------------------------------------------------------- ")  
  okGral = ok + okGral 
  NokGral = nok + NokGral
  y_tests.extend(tests)
  y_preds.extend(preds)


print("\n===========================================================================================================================")
print("\n= TOTAL GENERAL ", okGral+NokGral, ": Detectado OK ", okGral, "imágenes - Detectado con Error ", NokGral, "imágenes.\n\n")

 
print('\n= Exactitud: %f' % accuracy_score(y_tests, y_preds))
  
print('\n= Matriz de Confusión: ')
print(confusion_matrix(y_tests, y_preds, all_classes))

print("\n= Reporte de Clasificación: ")
print(classification_report(y_tests, y_preds))

print("\n===========================================================================================================================")
